In [0]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os
import sys
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
np.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.


In [0]:
def getFrames():
  video = cv2.VideoCapture("./000001_Short.mp4")
  total = 0
  total = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  print("Total : %d" %total)
  # video.set(1,82);

  # ret, frame = video.read()

  # cv2_imshow(frame)

  # cv2.waitKey()
  video.release()
  cv2.destroyAllWindows()

In [0]:
def reduceVideo():
  cap = cv2.VideoCapture("./000001.mp4")
  codec = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
  framerate=25
  resolution=(int(cap.get(3)),int(cap.get(4)))
  VideoFileOutput=cv2.VideoWriter("000001_Short.mp4",codec,framerate, resolution)
  count = 0
  while count < 100:
    count += 1
    success, frame = cap.read()
    if not success:
      break
    VideoFileOutput.write(frame)
  VideoFileOutput.release()        
  cap.release()
  cv2.destroyAllWindows()

In [0]:
def extractFeatures():
  model = VGG16(weights='imagenet', include_top=True)
  model.layers.pop()

  # model.summary()  

  # Load Yolo
  net = cv2.dnn.readNet("weights/yolov3-tiny.weights", "cfg/yolov3-tiny.cfg")
  # net = cv2.dnn.readNet("cfg/yolov3-tiny.cfg", "weights/yolov3-tiny.weights" )
  classes = []
  with open("coco.names", "r") as f:
      classes = [line.strip() for line in f.readlines()]
  layer_names = net.getLayerNames()
  output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
  colors = np.random.uniform(0, 255, size=(len(classes), 4))

  videos = os.listdir('./video/')
  ID = np.array([])
  labels = np.empty((0, 2))
  data = np.empty((0, 100, 20, 4096))
  det = np.empty((0, 100, 19, 4))

  for video_name in videos:
    print("Processing video %s" %video_name)
    ID = np.append(ID, np.array([video_name.split('.')[0]]))
    labels = np.vstack((labels, np.array([[0, 1]])))
    cap = cv2.VideoCapture('./video/' + video_name)
    det_temp = np.empty((0, 19, 4))
    frame_temp = np.empty((0, 20, 4096))
    frame_Count = 0
    while frame_Count < 100:
      frame_Count += 1
      border_temp = np.empty((0, 4))
      pixel_temp = np.empty((0, 4096))
      success, frame = cap.read()
      if not success:
        break
      height, width, channels = frame.shape
      # Detecting objects
      blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
      net.setInput(blob)
      outs = net.forward(output_layers)
      # Showing informations on the screen
      class_ids = []
      confidences = []
      boxes = []
      for out in outs:
          for detection in out:
              scores = detection[5:]
              class_id = np.argmax(scores)
              confidence = scores[class_id]
              if confidence > 0.10:
                  # Object detected
                  center_x = int(detection[0] * width)
                  center_y = int(detection[1] * height)
                  w = int(detection[2] * width)
                  h = int(detection[3] * height)
                  # Rectangle coordinates
                  x = int(center_x - w / 2)
                  y = int(center_y - h / 2)
                  boxes.append([x, y, w, h])
                  confidences.append(float(confidence))
                  class_ids.append(class_id)
      indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)

      img_path = 'temp.jpeg'
      cv2.imwrite(img_path, frame)      
      img = image.load_img(img_path, target_size=(224, 224))
      img_data = image.img_to_array(img)
      img_data = np.expand_dims(img_data, axis=0)
      img_data = preprocess_input(img_data)

      extract_fc1_features = Model(input=model.input, output=model.get_layer('fc1').output)
      fc1_features = extract_fc1_features.predict(img_data)
      fc1_features = np.squeeze(fc1_features)
      # print(fc1_features.shape)
      pixel_temp = np.vstack((pixel_temp, fc1_features))      

      # frame_crop = cv2.resize(frame, (64, 64))
      # frame_crop = cv2.cvtColor(frame_crop, cv2.COLOR_BGR2GRAY)
      # frame_crop = frame_crop / 255      
      # pixel_temp = np.vstack((pixel_temp, frame_crop.flatten()))

      for i in range(len(boxes)):
          if i in indexes:
              x1, y1, w, h = boxes[i]            
              x2 = x1 + w
              y2 = y1 + h
              border_temp = np.vstack((border_temp, np.array([[x1, y1, x2, y2]])))
              # cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)  
              crop = frame[y1:y2, x1:x2]
              cv2.imwrite(img_path, crop)      
              img = image.load_img(img_path, target_size=(224, 224))
              img_data = image.img_to_array(img)
              img_data = np.expand_dims(img_data, axis=0)
              img_data = preprocess_input(img_data)

              extract_fc1_features = Model(input=model.input, output=model.get_layer('fc1').output)
              fc1_features = extract_fc1_features.predict(img_data)
              fc1_features = np.squeeze(fc1_features)
              # print(fc1_features.shape)
              pixel_temp = np.vstack((pixel_temp, fc1_features))  

              # try:                
              #   crop = cv2.resize(crop, (64, 64))
              # except:
              #   cv2_imshow(frame)
              #   break
              # crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
              # crop = crop / 255
              # pixel_temp = np.vstack((pixel_temp, crop.flatten()))
      while border_temp.shape[0] < 19:
        border_temp = np.vstack((border_temp, np.zeros(4)))
      while pixel_temp.shape[0] < 20:
        pixel_temp = np.vstack((pixel_temp, np.zeros(4096)))
      key = cv2.waitKey(1)
      if key == 27:
          break    
      det_temp = np.vstack((det_temp, np.array([border_temp])))
      frame_temp = np.vstack((frame_temp, np.array([pixel_temp])))    
    cap.release()
    cv2.destroyAllWindows()  
    det = np.vstack((det, np.array([det_temp])))
    data = np.vstack((data, np.array([frame_temp])))  

    if data.shape[0] == 10:
      break
  
  return data, det, labels, ID
  

In [0]:
data, det, labels, ID = extractFeatures()
det = det.astype(int)

In [0]:
np.savez("data.npz", data=data, det=det, labels=labels, ID=ID)